In [ ]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [ ]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Recuperacion de pacientes

In [154]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
   select Id, IdentificationId__pc,Name from Account 
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name"]]
Pacientes_SF.shape

Empresas de Sales Force

In [ ]:

# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,  Name, Nombre_Empresa__c from Programas_por_Empresas__c
    """
)
programas_empresas = pd.DataFrame(results["records"])
programas_empresas = programas_empresas[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Name", "Nombre_Empresa__c", "Empresa_Padre__c"]
]
programas_empresas.shape

In [ ]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

In [ ]:
Empresas_SF_grande = pd.merge(
    left=programas_empresas,
    right=Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
Empresas_SF_grande.head()

In [ ]:
Empresas_SF_grande.shape

In [ ]:
archivo = 'empresas.xlsx'
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
#archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path("F:/Empresas SF/").joinpath(archivo_con_fecha)
print (hasta)
Empresas_SF_grande.to_excel(
    hasta, index=False
)

In [ ]:
results = sf.query_all(
    """
   select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c
    """
)
pacientes_programas = pd.DataFrame(results["records"])
pacientes_programas = pacientes_programas[
    ["Id", "Paciente__c", "Programas_por_Empresas__c"]
]
pacientes_programas.shape

In [ ]:
Empresas_SF_HL.head()

EMPRESAS_SF es la relacion de todas los Pacientes con las Empresas

In [ ]:
Empresas_SF = pd.merge(
    left=Empresas_SF,
    right=Empresas_SF_HL,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
Empresas_SF.head()

In [ ]:
Empresas_SF.shape

In [ ]:
empresas_EyDAtos = pd.merge(
    left=Empresas_SF,
    right=Empresas_SF_HL,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
empresas_EyDAtos.head()

Consulta de Recordtype son los codigos de atencion de las distintas ficvhas segunda pantalla ingreso derivacion esas cosas

In [ ]:
# Consulta de Servicios
results = sf.query_all(
    """
 select Id, Name from recordtype
    """
)
recordtype = pd.DataFrame(results["records"])
# Pacientes_empresas = Pacientes_empresas[
#   ["Id", "RUT__c",'CreatedDate','Name','Nombre_Empresa__c','Empresa__c','Tipo_de_Ficha__c','Account__c']
# ]

Pacientes_empresas.head()

In [ ]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_empresas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id",
)
Pacientes_con_ID.head()

In [ ]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_con_ID,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_y",
)
Pacientes_con_ID.head()

In [ ]:
results = sf.query_all(
    """
  select Id,RUT__c, CreatedDate, Clinica__c,Interacci_n__c, AppointmentNumber from ServiceAppointment 
    """
)
df = pd.DataFrame(results)
df.shape
citas_de_servicio = pd.DataFrame(results["records"])
citas_de_servicio = citas_de_servicio[
    ["RUT__c",'CreatedDate','Clinica__c','AppointmentNumber','Interacci_n__c']
]
citas_de_servicio.head()

In [ ]:
citas_de_servicio = pd.merge(
    left=citas_de_servicio,
    right=Pacientes_con_ID,
    how="left",
    left_on="RUT__c",
    right_on="IdentificationId__pc",
)
citas_de_servicio.head()

In [ ]:
#Nutricion
results = sf.query_all(
    """
 select Id,RUT__c, CreatedDate, Name, Nombre_Empresa__c, Empresa__c,Tipo_de_Ficha__c,Account__c,Motivo__c, Motivo_de_consulta__c,  RecordTypeId from Ficha_Clinica_Nutrici_n__c
    """
)

nutricion = pd.DataFrame(results["records"])
nutricion = nutricion[
    ["RUT__c",'CreatedDate','Name','Nombre_Empresa__c','Empresa__c','Tipo_de_Ficha__c','Account__c','Motivo__c','Motivo_de_consulta__c','RecordTypeId']
]

nutricion.head()

In [ ]:
nutricion = pd.merge(
    left=nutricion,
    right=Pacientes_con_ID,
    how="left",
    left_on="RUT__c",
    right_on="IdentificationId__pc",
)
nutricion.head()

In [ ]:
Empresas_SF.to_excel(
    "C:/Users/jmartinez/Documents/cargas/empresas.xlsx", index=False
)